# On-Device AI Co., Ltd.

Web   : https://on-device-ai.com/  
Email : yilintung@on-device-ai.com

https://github.com/MIT-LCP/wfdb-python  
https://github.com/PIA-Group/BioSPPy

In [1]:
import numpy as np
import wfdb
from biosppy.signals import ecg

In [2]:
# MIT-BIH Arrhythmia Database
# https://physionet.org/content/mitdb/1.0.0/
mitdbs = [100,101,102,103,104,105,106,107,108,109,111,112,113,114,115,116,117,118,119,121,122,123,124,200,201,202,203,205,207,208,209,210,212,213,214,215,217,219,220,221,222,223,228,230,231,232,233,234]

In [3]:
# Beat annotations
# https://archive.physionet.org/physiobank/annotations.shtml
beat_annotations = ['N', 'L', 'R', 'B', 'A', 'a',    'J', 'S', 'V', 'r', 'F', 'e', 'j', 'n', 'E', '/',    'Q',     '?'    ]

In [4]:
# Beat annotations to AAMI EC57 categories
aami_normal = ['N','L','R', 'e','j']
aami_supraventricular_ectopic_beat = ['A','a', 'J', 'Q']
aami_ventricular_ectopic_beat = ['V','E']
aami_fusion_beat = ['F']
aami_unknown_beat = ['/','f','u']

In [5]:
for db in mitdbs :

    # Read a WFDB record
    sig, fields = wfdb.rdsamp('./mitdb/'+str(db))
    
    # Only using Lead II signal
    if fields['sig_name'][0] == 'MLII' :
        signal = sig[:,0]
    elif fields['sig_name'][1] == 'MLII' :
        signal = sig[:,1]
    else :
        signal = None

    # Read a WFDB annotation
    ann =  wfdb.rdann('./mitdb/'+str(db), 'atr')
    annsample = ann.sample
    annsymbol = ann.symbol

    if signal is not None :
        
        # Convert signal (numpy array) type and values from Float64 to Float32
        signal = np.float32(signal)
        
        # ECG R-peak segmentation algorithm.
        # Follows the approach by P.S. Hamilton, “Open Source ECG Analysis Software Documentation”, E.P.Limited, 2002.
        out = ecg.hamilton_segmenter(signal=signal, sampling_rate=fields['fs'])
        rpeaks = out['rpeaks']

        count                = 0
        signal_size          = len(signal)
        ann_data_size        = len(annsample)
        beat_list            = list()
        while count < ann_data_size:
            sample = annsample[count]
            symbol = annsymbol[count]

            if symbol in beat_annotations:

                # Find the R Peak for the beat annotation
                beat_rpeak        = None
                check_rpeak_start = sample - 90
                check_rpeak_end   = sample + 90
                if check_rpeak_start >= 0 and check_rpeak_end < signal_size :
                    for rpeak in rpeaks :
                        if rpeak >= check_rpeak_start and rpeak <= check_rpeak_end :
                            beat_rpeak = rpeak
                            break

                # If found R Peak, set ECG heartbeat segmentation
                if beat_rpeak is not None :

                    segmentation_start = beat_rpeak - 90
                    segmentation_end   = beat_rpeak + 170
                    if segmentation_start >= 0 and segmentation_end < signal_size :

                        # Segmentation : R Peak - 90 ~ R Peak + 170
                        segmentation = signal[segmentation_start:segmentation_end]

                        # z-score normalization
                        segmentation_copy = np.copy(segmentation)
                        normalized_zone = (segmentation_copy - np.mean(segmentation_copy))/np.std(segmentation_copy)

                        # AAMI categories : ANSI/AAMI EC57; 2012
                        # Class N SVEB VEB F Q
                        # id    0 1    2   3 4
                        if   symbol in aami_normal :
                            aami_id = 0
                        elif symbol in aami_supraventricular_ectopic_beat :
                            aami_id = 1
                        elif symbol in aami_ventricular_ectopic_beat :
                            aami_id = 2
                        elif symbol in aami_fusion_beat :
                            aami_id = 3
                        elif symbol in aami_unknown_beat :
                            aami_id = 4
                        else :
                            aami_id = 4
                        
                        # Add to list : [0] AAMI categorie , [1] Normalized signal
                        beat_list.append([aami_id,normalized_zone])
            # Next annotation
            count += 1

        # Save Numpy array
        np.save('./beats/'+ str(db) + '.npy',{'beats':beat_list})